# EcoCommons Notebook: MaxEnt demonstration

---
**Author details:** EcoCommons Platform   
**Contact details:** comms@ecocommons.org.au  
**Date:** April 2023    
**Copyright statement:** This script is the product of EcoCommons platform.   
                      Please refer to the <a href="https://www.ecocommons.org.au/">EcoCommons website</a> for more details.

  
**Script and data info**:&nbsp;   
 In this notebook, you will run a Maximum Entropy Modeling (MaxeEnt) to predict 
 occurrences by finding the most uniform distribution within the limits of
 the predictor variables. Maxent is an open-source software created by Phillips and collaborators. More info can be found on <a href="https://biodiversityinformatics.amnh.org/open_source/maxent/">the MaxEnt website.</a>  

For those of you new to R, find tips on how to get started <a href="https://datacarpentry.org/R-ecology-lesson/">can be found here</a>.


**In this example you will:**
   1. Set up your working environment
   2. Import and clean occurrence data
   3. Import bioclimatic variables data from 'netCDF' files
   4. Train a MaxEnt model    
   
---

In [ ]:
# Load packages that are included in the R environment
library(dismo)
library(jpeg)
library(maps)
library(raster)
library(rasterVis)
library(readxl)
library(rgbif)
library(rgeos)
library(rJava)
library(sp)
library(svMisc)
library(rgdal)
library(terra)
library(tidyr)

- Tip: you can create a personal library for installing packages that will persist between sessions
- You only need to create the lib directory once, but each new session you may need to set `.libPaths()` again to load previously installed packages. 
- Warning: Packages you install into the default library path will not persist between sessions

In [ ]:
# Check the default library paths
.libPaths()

In [ ]:
# Create a personal library in your chosen location or under the current working directory
dir.create("./lib")

In [ ]:
# Prepend the path to your personal library to .libPaths
# This will add your library path (which will be the default path for installing new packages)
# But also keep the default R environment path (to `library` existing packages)
.libPaths(c("./lib", .libPaths()))

In [ ]:
install.packages("ncdf4")
library(ncdf4)

# 1. Set up your environment
---

We will work inside the current directory of this example notebook. A folder for `data` already exists, but we will create an `output` folder as well. 

In [ ]:
dir.create("./output")

Check if the MaxEnt `.jar` file is installed and and available in our environment. 

In [ ]:
jar <- paste(system.file(package = "dismo"), "/java/maxent.jar", sep = '') 
if (file.exists(jar)) {
  cat("MaxEnt is available.")
  } else {
    cat('MaxEnt is not available!')
}

# 2. Import and filter occurrence data
---
- We will download occurrence records of the frog *Litoria fallax* (Eastern Dwarf Tree Frog), from the GBIF database.
- Warning: setting a high limit here can run slowly. So for this example, we limit the query to 500 records.
- For large queries, we recommend downloading directly from GBIF, or using the `occ_download*()` functions which are more appropriate for larger requests. 

In [ ]:
# Select your species
my_species <- c("Litoria fallax")

# Download GBIF occurrence data
Litoria_fallax_GBIF_raw <- rgbif:: occ_data(scientificName = my_species,
                                            hasCoordinate  = TRUE,
                                            limit          = 500) 

Get the information needed to cite this data.

In [ ]:
# Return a list of citations for the downloaded GBIF data: 
citations <- rgbif::gbif_citation(Litoria_fallax_GBIF_raw)

# Example of a citation:
print(citations[[1]])

- Get familiar with the data, and check how the data is organized using `str()`

In [ ]:
str(Litoria_fallax_GBIF_raw, list.len = 4)

- Explore the `$data` element, which contains the data frame

In [ ]:
View(head(Litoria_fallax_GBIF_raw$data))

- Select columns needed for mapping and cleaning the occurrence data. 
- This is just an example of columns that you could select.
- We will also filter the records

In [ ]:
# Select columns, and return a `tibble` with only unique rows.
litoria_fallax <- unique(tibble::as_tibble(Litoria_fallax_GBIF_raw$data [ , c("decimalLongitude", "decimalLatitude",
                                                     "individualCount", "species", "year",
                                                     "month", "country", "occurrenceStatus",
                                                     "coordinateUncertaintyInMeters", "datasetName",
                                                     "datasetKey")]))

cat("- There are", nrow(litoria_fallax), "unique occurrence records in the tibble.\n")

In [ ]:
# You could subset by a chosen dataset
litoria_fallax <- litoria_fallax[litoria_fallax$datasetName == "iNaturalist research-grade observations",]
# Include only records with a particualary uncertainty
litoria_fallax <- litoria_fallax[litoria_fallax$coordinateUncertaintyInMeters < 200,]
# Drop NAs
litoria_fallax <- litoria_fallax[!is.na(litoria_fallax$datasetName), ]

cat("- There are now", nrow(litoria_fallax), "occurrence records in the tibble after filtering.\n")

In [ ]:
View(head(litoria_fallax))

- We can export the `tibble` as a `.csv` file

In [ ]:
write.csv(litoria_fallax, paste0(getwd(),"/data/Litoria_fallax_filtered.csv"))

### Map the occurrence data  
When downloading data from big datasets, some records are likely to be centroids of(relatively large) grid cells on which particular surveys were based. You can notice them when distribution records are too regularly spaced to be exact locations of species sightings. When it happens, remember to adjust the spatial resolution of your analysis accordingly.

In [ ]:
# NOTE: added cex (point size) and col so the records are easier to see
map("world", xlim = range(litoria_fallax$decimalLongitude),
    ylim = range(litoria_fallax$decimalLatitude))  
points(litoria_fallax[ , c("decimalLongitude", "decimalLatitude")], pch = ".", cex = 3, col = "blue")


# 3. Load and process the bioclimatic data
---

It is example, we will be using data from a netCDF file, obtained from "TerraClimate", a dataset of monthly climate and climatic water balance for global terrestrial surfaces from 1958-2019. We decided to include four climatic variables (maximum and minimum temperature, rainfall and soil cover - called "tmax", "tmin", "ppt" and "soil", respectively).

In [ ]:
# Read NetCDF data file of predictors
file <- "~/educational_material/Maxent_example/data/Terraclim_EY_E_Aus_orig.nc"
var_names <- c("tmax", "tmin", "ppt", "soil")  

# Visualise one of the variables (tmax)
plot(raster::brick(file, varname = "tmax"))

In [ ]:
# Save a CDF file of the mean values for each of the variables, in your directory folder
for (var_name in var_names) {
  
  var_brick <- raster::brick(file, varname = var_name)
  var_mean  <- mean(var_brick)
  
  raster::writeRaster(x         = var_mean, 
                      filename  = paste0("./data/",var_name, "_mean"),
                      overwrite = TRUE, 
                      format    = 'CDF')
  }

Stack the predictors together and rename them. Plot to see the result.

In [ ]:
mean_files <- list.files("./data", pattern = "_mean.nc", full.names = TRUE)
predictors <- raster::stack(mean_files)
names(predictors) <- c('Rain_mean', 'Soil_mean', 'MXtemp_mean', 'MNtemp_mean')
plot(predictors)

In [ ]:
xy_fallax <- litoria_fallax[, c("decimalLongitude", "decimalLatitude")]
colnames(xy_fallax) <- c("x", "y")
xy_fallax_sp <- sp::SpatialPoints(coords = xy_fallax, proj4string= CRS("+proj=longlat +datum=WGS84 +no_defs"))

Remove distribution records that are outside of your area of interest and plot to see the result.

In [ ]:
xy_fallax_sp <- raster::crop(xy_fallax_sp, predictors[[1]])

plot(predictors[[1]])
points(xy_fallax_sp)

# 4. Run a MaxEnt model
---

### Prepare data and fit model

First, create training and test points.

In [ ]:
group <- dismo::kfold(xy_fallax_sp, k = 5)
pres_test  <- xy_fallax_sp[group == 1, ]  # 20% of data sample for testing
pres_train <- xy_fallax_sp[group != 1, ]

Set the background points and check NAs.

In [ ]:
backg <- dismo::randomPoints(mask = predictors, n = 1000)
colnames(backg) <- c("lon", "lat")

# check the number of NAs
x <- raster::extract(predictors, pres_train) 
y <- na.omit(x)
na_count <- length(y)/length(x)  # need to be 0.5 or more
na_count

### Set the various Maxent arguments

A full list of the available arguments are at the end of the notebook.  Other things to consider, are adding a bias layer.

In [ ]:
maxent_args <- c("removeduplicates=TRUE","jackknife=TRUE")

### Run the model

- There are different ways to run MaxEnt in R. We choose to use the 'dismo' package.
- The outputs will be saved in the `output` folder of the current working directory
- This will include the MaxEnt `.html` summary file

In [ ]:
basic_maxent<- dismo::maxent(predictors,
                             pres_train,
                             path = "./output",
                             args = maxent_args)

In [ ]:
plot(basic_maxent)

Create a RasterLayer with the prediction.

In [ ]:
map_predictions <- dismo::predict(basic_maxent, predictors)

In [ ]:
plot(map_predictions)
points(pres_train)

Use the 'evaluate' function on 'dismo' package to see some key results.

In [ ]:
evaluate_model <- dismo::evaluate(pres_train, pres_test, basic_maxent, predictors)
evaluate_model

In [ ]:
evaluate_model@auc

In [ ]:
# Save the prediction plot with training points as a `jpeg`
jpeg("./output/max_prediction.jpeg")
plot(map_predictions)
points(pres_train)
dev.off()


In [ ]:
# Save the prediction in an `asc` file

raster::writeRaster(map_predictions,
                    filename  = "./output/Litoria_fallax_pred.asc",
                    format    = "ascii",
                    overwrite = TRUE)

Find a threshold (cut-off) to transform model predictions probabilities to a binary score (presence or absence).

In [ ]:
threshold_model <- dismo::threshold(evaluate_model, 'spec_sens')
threshold_model

Now, it is time to reclasify the values of the raster object. This is step is OPTIONAL.

In [ ]:
m <- c(0, threshold_model, 0,  threshold_model, 1, 1)
reclass <- matrix(m, ncol = 3, byrow = TRUE)
rc <- raster::reclassify(map_predictions, reclass)

In [ ]:
jpeg("./output/pres_absence_map.jpeg")
plot(rc, main = 'presence/absence')
points(pres_train, pch = '+')
dev.off()

Generate response plots, single variable response curves for the model you run.

In [ ]:
dismo::response(basic_maxent)

- Create a data frame with the evaluation results.

In [ ]:
myspecies <- c("Litoria fallax")
cor <- unname(evaluate_model@cor)
test_data_results <- as.data.frame(list(myspecies,
                                        evaluate_model@np,
                                        evaluate_model@na,
                                        evaluate_model@auc,
                                        cor))

In [ ]:
colnames(test_data_results) <- c("species", "presences", "absences", "AUC", "cor")
test_data_results

Another option to visualize the results.

In [ ]:
basic_maxent@results

# 5. List of other MaxEnt arguments you could use

In [ ]:
maxent_args <- c(
  
  #duplicate records
  'removeduplicates=TRUE', #remove duplicate presence records. If environmental data are in grids, duplicates are records in the same grid cell, otherwise, duplicates are records with identical coordinates.
  
  #background records
  'maximumbackground=10000', #if the number of background points/grid cells is larger than this number, then this number of cells is chosen randomly for background points.
  'addsamplestobackground=TRUE', #add to the background any sample for which has a combination of environmental values that isn't already present in the background
  'addallsamplestobackground=FALSE', #add all samples to the background, even if they have combinations of environmental values that are already present in the background
  
  #missing data
  'allowpartialdata=FALSE', #during model training, allow use of samples that have nodata values for one or more environmental variables
  
  #variable importance
  'jackknife=TRUE', #NB: default=FALSE; measure importance of each environmental variable by training with each environmental variable first omitted, then used in isolation.
  
  #random seed
  'randomseed=FALSE', #if selected, a different random seed will be used for each run, so a different random test/train partition will be made and a different random subset of the background will be used, if applicable.
  
  #prevalence
  'defaultprevalence=0.5', #default prevalence of the species: probability of presence at ordinary occurrence points. See Elith et al. Diversity and Distributions, 2011 for details
  
  #train/test settings
  'randomtestpoints=0', #percentage of presence localities to be randomly set aside as test points, used to compute AUC, omission, etc.
  'replicates=1', #number of replicate runs to do when cross-validating, bootstrapping or doing sampling with replacement runs.
  'replicatetype=crossvalidate', #if replicates > 1, do multiple runs of this type: 
      #crossvalidate: samples divided into replicates fods; each fold in turn used for test data
      #bootstrap: replicate sample sets chosen by sampling with replacement
      #subsample: replicate sample sets chosen by removing random test percentage without replacement to be used for evaluation
  'maximumiterations=500', #stop training after this many iterations of the optimization algorithm
  'convergencethreshold=0.00001', #stop training when the drop in log loss per iteration drops below this number 
  
  #feature selection
  'autofeature=TRUE', #automatically select which feature classes to use, based on number of training samples
  'linear=TRUE', #allow linear features to be used
  'quadratic=TRUE', #allow quadratic features to be used
  'product=TRUE', #allow product features to be used
  'threshold=TRUE', #allow threshold features to be used
  'hinge=TRUE', #allow hinge features to be used
  
  #feature settings
  'lq2lqptthreshold=80', #number of samples at which product and threshold features start being used
  'l2lqthreshold=10', #number of samples at which quadratic features start being used
  'hingethreshold=15', #number of samples at which hinge features start being used
  
  #regularization settings
  'betamultiplier=1', #multiply all automatic regularization parameters by this number. A higher number gives a more spread-out distribution.
  'beta_threshold=-1', #regularization parameter to be applied to all threshold features; negative value enables automatic setting
  'beta_categorical=-1', #regularization parameter to be applied to all categorical features; negative value enables automatic setting
  'beta_lqp=-1', #regularization parameter to be applied to all linear, quadratic and product features; negative value enables automatic setting
  'beta_hinge=-1', #regularization parameter to be applied to all hinge features; negative value enables automatic setting
  
  #outputs - NB. These are not shown in the UI, so unable to be changed by user
  
  'responsecurves=TRUE', #NB. default=FALSE; create graphs showing how predicted relative probability of occurrence depends on the value of each environmental variable
  'responsecurvesexponent=FALSE', #instead of showing the logistic value for the y axis in response curves, show the exponent (a linear combination of features).
  'pictures=TRUE', #create a .png image for each output grid
  'outputformat=raw', #representation of probabilities used in writing output grids, see Help for details
  'writeclampgrid=TRUE', #write a grid that shows the spatial distribution of clamping. At each point, the value is the absolute difference between prediction values with and without clamping.
  'writemess=TRUE', #a multidimensional environmental similarity surface (MESS) shows where novel climate conditions exist in the projection layers. The analysis shows both the degree of novelness and the variable that is most out of range at each point.
  'writeplotdata=FALSE', #write output files containing the data used to make response curves, for import into external plotting software.
  'outputgrids=TRUE', #write output grids. Turning this off when doing replicate runs causes only the summary grids (average, std, deviation, etc) to be written, not those for the individual runs.
  'plots=TRUE', #write various plots for inclusion in .html output
  'logfile=maxent.log', #file name to be used for writing debugging information about a run in output directory
  #'applythresholdrule=Fixed cumulative value 1', #apply a threshold rule, generating a binary outputgrid in addition to the regular prediction grid. Use the full name of the threshold rule in Maxent's html output as the argument. For example 'applyThresholdRule=Fixed cumulative value 1'.
  'logscale=TRUE', #if selected, all pictures of models will use a logarithmic scale for color-coding
  'writebackgroundpredictions=FALSE', #write .csv file with predictions at background points
  'fadebyclamping=FALSE', #reduce prediction at each point in projections by the difference between clamped and non-clamped output at that point

  #projection settings NB. These are not shown in the UI, so unable to be changed by user
  
  'extrapolate=TRUE', #predict to regions of environmental space outside the limits encountered during training
  'doclamp=TRUE' #apply clamping when projecting
  
)